### Imports

In [4]:
!pip install datasets
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qudta1k1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qudta1k1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [1]:
import torch
from torch import nn
from datasets import load_dataset
from diffusers import StableDiffusionPipeline
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
import numpy as np
from PIL import Image
import clip
from tqdm import tqdm

### Data preparation

In [79]:
num_samples = 10
dataset = load_dataset("vinesmsuic/GenAI-Bench_image_edition_processed")
dataset = dataset['train']

def get_preferred_image(row):
    if row['vote_type'] == 'rightvote':
        return row['right_output_image']
    else:
        return row['left_output_image']

def is_valid_size(image, size=(512, 512)):
    """Check if image has the required dimensions"""
    return image.size == size

valid_indices = []
for idx, item in enumerate(dataset):
    source_img = item['source_image']
    preferred_img = get_preferred_image(item)

    if is_valid_size(source_img) and is_valid_size(preferred_img):
        valid_indices.append(idx)

dataset = dataset.select(valid_indices)
dataset = dataset.select(range(num_samples))

dataset = {
    'source_image': dataset['source_image'],
    'target_prompt': dataset['target_prompt'],
    'source_prompt': dataset['source_prompt'],
    'instruct_prompt': dataset['instruct_prompt'],
    'target_image': [get_preferred_image(row) for row in dataset],
    'vote_type': dataset['vote_type'],
    'preference': dataset['preferences']
}

### Model Setup

In [80]:
model_name = "CompVis/stable-diffusion-v1-4"
stable_diffusion = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda")

clip_model, clip_preprocess = clip.load("ViT-B/32", device="cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### Baseline Generation and Evaluation

In [81]:
generated_images = []

for prompt, src_image in tqdm(zip(dataset['target_prompt'], dataset['source_image']), total=len(dataset['target_prompt'])):
    images = stable_diffusion(
        prompt,
        heihht=src_image.size[0],
        width=src_image.size[1],
        num_inference_steps=50,
        guidance_scale=7.5,
        progress_bar=False).images
    generated_images.extend(images)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:07<01:09,  7.69s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:15<01:01,  7.68s/it]

...

 80%|████████  | 8/10 [01:03<00:15,  7.98s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [01:11<00:07,  7.96s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:18<00:00,  7.89s/it]


In [82]:
def compute_ssim(img1, img2):
    """Compute SSIM between two images"""
    # Convert PIL images to numpy arrays
    img1_np = np.array(img1.convert('L'))
    img2_np = np.array(img2.convert('L'))

    return ssim(img1_np, img2_np)

In [83]:
def compute_clip_similarity_img2img(clip_model, clip_preprocess, img1, img2):
    """Compute CLIP similarity between images or image and text"""
    # Image to image clip similarity
    img1_processed = clip_preprocess(img1).unsqueeze(0).to("cuda")
    img2_processed = clip_preprocess(img2).unsqueeze(0).to("cuda")

    with torch.no_grad():
        img1_features = clip_model.encode_image(img1_processed)
        img2_features = clip_model.encode_image(img2_processed)
        similarity = torch.cosine_similarity(img1_features, img2_features).item()

    return similarity

def compute_clip_similarity_img2text(clip_model, clip_preprocess, img, text):
    # Image to text clip similarity
    img_processed = clip_preprocess(img).unsqueeze(0).to("cuda")
    text_tokens = clip.tokenize([text]).to("cuda")
    with torch.no_grad():
        img_features = clip_model.encode_image(img_processed)
        text_features = clip_model.encode_text(text_tokens)
        similarity = torch.cosine_similarity(img_features, text_features).item()

    return similarity

In [84]:
metrics = {
    'ssim_scores': [],
    'clip_img2img_target_scores': [],
    'clip_img2img_source_scores': [],
    'clip_img2text_target_scores': [],
    'clip_img2text_source_scores': [],
}

for gen_img, source_img, source_prompt, target_img, target_prompt in zip(generated_images, dataset['source_image'], dataset['source_prompt'], dataset['target_image'], dataset['target_prompt']):
    # Compute SSIM
    ssim_score = compute_ssim(gen_img, target_img)
    metrics['ssim_scores'].append(ssim_score)

    # Compute CLIP similarities
    metrics['clip_img2img_target_scores'].append(compute_clip_similarity_img2img(clip_model, clip_preprocess, gen_img, target_img))
    metrics['clip_img2img_source_scores'].append(compute_clip_similarity_img2img(clip_model, clip_preprocess, gen_img, source_img))
    metrics['clip_img2text_target_scores'].append(compute_clip_similarity_img2text(clip_model, clip_preprocess, gen_img, target_prompt))
    metrics['clip_img2text_source_scores'].append(compute_clip_similarity_img2text(clip_model, clip_preprocess, gen_img, source_prompt))

In [85]:
for metric_name, scores in metrics.items():
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    print(f"{metric_name}:\t{mean_score:.4f} (±{std_score:.4f})")

ssim_scores:	0.1863 (±0.0703)
clip_img2img_target_scores:	0.7277 (±0.1424)
clip_img2img_source_scores:	0.6639 (±0.1376)
clip_img2text_target_scores:	0.3234 (±0.0260)
clip_img2text_source_scores:	0.2908 (±0.0249)
